# HAMOYE STAGE D (TAG ALONG CODE) 
# SUBMITTED BY -> APURV DESHPANDE

### INTRODUCTION : Every minute, the world loses an area of forest the size of 48 football fields. And deforestation in the Amazon Basin accounts for the largest share, contributing to reduced biodiversity, habitat loss, climate change, and other devastating effects. But better data about the location of deforestation and human encroachment on forests can help governments and local stakeholders respond more quickly and effectively.

IMPORTING THE REQUIRED LIBRARIES

In [7]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import gc
import numpy as np
from keras import backend as K
from sklearn.metrics import fbeta_score
from keras.layers import Conv2D, Dense, LSTM, Flatten, MaxPooling2D, BatchNormalization, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.metrics import fbeta_score
from tqdm import tqdm
from sklearn.utils import shuffle
import cv2
from PIL import Image
from tensorflow import keras
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential 
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, History

THE DATASET REQUIRED FOR THIS NOTEBOOK CONTAIN THE BELOW FILES:
1. train.csv - a list of training file names and their labels, the labels are space-delimited
2. sample_submission.csv - correct format of submission, contains all the files in the test set.
3. [train/test]-tif-v2.tar.7z - tif files for the training/test set 
4. [train/test]-jpg[-additional].tar.7z - jpg files for the trainin/test set 
5. Kaggle-planet-[train/test]-tif.torrent - a BitTorrent file for downloading [train/test]-tif-v2.tar.7z 

In [8]:
#reading the labels
train_label = pd.read_csv('../input/planets-dataset/planet/planet/train_classes.csv')
sam_sub = pd.read_csv('../input/planets-dataset/planet/planet/sample_submission.csv')
train_label.head()

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary
3,train_3,clear primary
4,train_4,agriculture clear habitation primary road


In [9]:
#onehot=OneHotEncoder()
encoder = LabelEncoder()

In [10]:
#encoder.fit(train_label['tags'])
label_maps = pd.DataFrame()
label_maps['tags'] = ['agriculture', 'artisinal_mine', 'bare_ground','blooming','blow_down','clear','cloudy','conventional_mine','cultivation','habitation','haze', 'partly_cloudy','primary','road','selective_logging','slash_burn','water']
label_maps['map'] = encoder.fit_transform(label_maps['tags'])
label_maps

,tags,map
0,agriculture,0
1,artisinal_mine,1
2,bare_ground,2
3,blooming,3
4,blow_down,4
5,clear,5
6,cloudy,6
7,conventional_mine,7
8,cultivation,8
9,habitation,9


In [11]:
#defining a dict of encoded labels
label_map = {'agriculture': 0,
 'artisinal_mine': 1,
 'bare_ground': 2,
 'blooming': 3,
 'blow_down': 4,
 'clear': 5,
 'cloudy': 6,
 'conventional_mine': 7,
 'cultivation': 8,
 'habitation': 9,
 'haze': 10,
 'partly_cloudy': 11,
 'primary': 12,
 'road': 13,
 'selective_logging': 14,
 'slash_burn': 15,
 'water': 16}


In [12]:
#loading the traing_images
X = []
Y = []
train_label = shuffle(train_label,random_state=0)

In [13]:
for image_name, tags in tqdm(train_label.values, miniters=400):
    arr = cv2.imread('../input/planets-dataset/planet/planet/train-jpg/{}.jpg'.format(image_name), cv2.IMREAD_UNCHANGED)
    targets = np.zeros(17)
    for t in tags.split(' '):
      targets[label_map[t]] = 1 
    arr = cv2.resize(arr, (64, 64))
    X.append(arr)
    Y.append(targets)

100%|██████████| 40479/40479 [06:04<00:00, 111.00it/s]


In [14]:
X = np.array(X, np.float16)/255.0

In [15]:
#splitting
X = np.array(X)
Y = np.array(Y)
x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size = 0.2, shuffle = True, random_state = 1)

print(x_train.shape, y_train.shape, x_val.shape, y_val.shape)

(32383, 64, 64, 3) (32383, 17) (8096, 64, 64, 3) (8096, 17)


In [16]:
del X, Y
gc.collect()

69

In [17]:
gc.collect()

23

In [18]:
def fbeta(y_true, y_pred, threshold_shift=0):
    beta = 2

    # just in case of hipster activation at the final layer
    y_pred = K.clip(y_pred, 0, 1)

    # shifting the prediction threshold from .5 if needed
    y_pred_bin = K.round(y_pred + threshold_shift)

    tp = K.sum(K.round(y_true * y_pred_bin)) + K.epsilon()
    fp = K.sum(K.round(K.clip(y_pred_bin - y_true, 0, 1)))
    fn = K.sum(K.round(K.clip(y_true - y_pred, 0, 1)))

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)

    beta_squared = beta ** 2
    return (beta_squared + 1) * (precision * recall) / (beta_squared * precision + recall + K.epsilon())

In [20]:
from tensorflow.keras.optimizers import Adam

In [21]:
model = keras.Sequential()
model.add(Conv2D(64, 5, 2, activation = "relu", input_shape = (64, 64, 3)))
model.add(MaxPooling2D())
model.add(Conv2D(128, 5, 2, activation = "relu"))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(512, activation = "relu"))
model.add(Dense(17, activation = "sigmoid"))
model.compile(loss = "binary_crossentropy", optimizer = Adam(), metrics = [fbeta])
model.fit(x_train, y_train, validation_data = (x_val, y_val), epochs = 45, batch_size = 64)

2023-01-11 06:30:09.500727: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/45
506/506 [==============================] - 36s 69ms/step - loss: 0.2244 - fbeta: 0.6343 - val_loss: 0.2034 - val_fbeta: 0.6898
Epoch 2/45
506/506 [==============================] - 34s 67ms/step - loss: 0.1949 - fbeta: 0.6878 - val_loss: 0.1841 - val_fbeta: 0.6887
Epoch 3/45
506/506 [==============================] - 34s 68ms/step - loss: 0.1772 - fbeta: 0.7209 - val_loss: 0.1699 - val_fbeta: 0.7390
Epoch 4/45
506/506 [==============================] - 34s 68ms/step - loss: 0.1654 - fbeta: 0.7437 - val_loss: 0.1581 - val_fbeta: 0.7572
Epoch 5/45
506/506 [==============================] - 34s 68ms/step - loss: 0.1564 - fbeta: 0.7590 - val_loss: 0.1591 - val_fbeta: 0.7606
Epoch 6/45
506/506 [==============================] - 34s 67ms/step - loss: 0.1520 - fbeta: 0.7675 - val_loss: 0.1471 - val_fbeta: 0.7877
Epoch 7/45
506/506 [==============================] - 34s 67ms/step - loss: 0.1478 - fbeta: 0.7740 - val_loss: 0.1510 - val_fbeta: 0.7570
Epoch 8/45
506/506 [==============

In [22]:
gc.collect()

2300

In [23]:
#with tpu_strategy.scope():
test_loss, test_accuracy = model.evaluate(x_val, y_val)
print('Test loss: {}'.format(test_loss))
print('Test accuracy: {}'.format(test_accuracy))

253/253 [==============================] - 4s 15ms/step - loss: 0.2005 - fbeta: 0.8086
Test loss: 0.20054876804351807
Test accuracy: 0.8085652589797974


In [24]:
gc.collect()

790

In [ ]:
model.evaluate(x_val, y_val)

print("train fscore: ", fbeta_score(y_train, np.round(model.predict(x_train)), 2,average = 'weighted'))
print("val fscore: ", fbeta_score(y_val, np.round(model.predict(x_val)), 2, average = 'weighted'))

253/253 [==============================] - 1s 2ms/step - loss: 0.2290 - fbeta: 0.8059
        
train fscore:  0.9321623395315798
    
val fscore:  0.800504961104014

In [29]:
del x_train, y_train, x_val, y_val
gc.collect()

3133

In [30]:
#dividing my test_labels into two part for test-jpg and test-jpg-additional
test = sam_sub[0 : 40669]
files = sam_sub[40669 : ]

In [31]:
#with tpu_strategy.scope():   
test_img = []

for image_name, tags in tqdm(test.values, miniters=1000):
    arr = cv2.imread('../input/planets-dataset/planet/planet/test-jpg/{}.jpg'.format(image_name))
    test_img.append(cv2.resize(arr, (64, 64)))

for image_name, tags in tqdm(files.values, miniters=1000):
    arr = cv2.imread('../input/planets-dataset/test-jpg-additional/test-jpg-additional/{}.jpg'.format(image_name))
    test_img.append(cv2.resize(arr, (64, 64)))

test_img = np.array(test_img, np.float16)/255.0

100%|██████████| 20522/20522 [02:59<00:00, 114.40it/s]


In [32]:
gc.collect()

69

In [33]:
#with tpu_strategy.scope():
yres = []
predictions = model.predict(test_img, batch_size = 64, verbose = 2)
yres.append(predictions)

957/957 - 21s


In [34]:
gc.collect()

790

In [35]:
#converting my encoded labels back to it original form
sub = np.array(yres[0])
for i in range (1, len(yres)):
    sub += np.array(yres[i])
sub = pd.DataFrame(sub, columns = label_map)


In [36]:
sub

,agriculture,artisinal_mine,bare_ground,blooming,blow_down,clear,cloudy,conventional_mine,cultivation,habitation,haze,partly_cloudy,primary,road,selective_logging,slash_burn,water
0,0.004219,9.889225e-13,2.206862e-04,2.118737e-02,1.887381e-04,9.989491e-01,1.135123e-05,1.801879e-13,2.438724e-04,7.297299e-05,1.350582e-03,5.357483e-06,0.999980,1.599818e-03,4.145950e-03,6.307321e-11,4.451305e-03
1,0.002653,5.361746e-17,3.896654e-04,7.783175e-04,6.388382e-05,9.936229e-01,1.405802e-08,3.087305e-11,2.954602e-04,3.780127e-04,6.343905e-05,5.857408e-03,1.000000,2.200112e-02,3.991872e-03,6.127926e-12,6.642133e-03
2,0.000051,5.406998e-16,1.381482e-14,4.329932e-18,3.563331e-09,6.970959e-07,5.524964e-14,2.013242e-07,1.830420e-08,1.951057e-19,7.678187e-15,1.000000e+00,1.000000,5.265280e-08,7.031927e-18,1.398157e-16,9.098262e-07
3,0.086665,8.539542e-10,4.958808e-04,4.948497e-03,6.805956e-04,3.467108e-01,4.070355e-08,1.448375e-10,5.220497e-02,4.137248e-03,3.930032e-04,6.606513e-01,1.000000,1.670182e-03,1.661122e-03,1.498179e-09,7.572472e-04
4,0.013722,1.070724e-17,1.241287e-14,2.297222e-28,2.432174e-24,3.419948e-06,1.032323e-03,7.675015e-10,6.526513e-09,4.514588e-09,6.884259e-11,9.953701e-01,0.978547,6.394982e-04,2.168931e-16,1.372165e-13,9.994837e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61186,0.000078,5.053287e-19,1.247695e-09,4.250575e-18,9.978343e-26,1.107603e-03,6.905466e-03,2.485451e-14,2.500207e-09,8.856267e-03,1.590921e-06,3.067651e-02,0.143395,9.569941e-01,1.969561e-15,4.591059e-19,1.713467e-02
61187,0.935881,3.000153e-15,2.102352e-07,1.762059e-14,1.340426e-16,9.999914e-01,1.197885e-07,1.015662e-09,4.421785e-01,3.353119e-03,1.536248e-06,6.849113e-09,0.999992,1.975000e-04,2.229045e-10,7.566009e-05,9.999119e-01
61188,0.089203,1.934194e-15,8.324797e-06,7.793427e-03,4.502271e-05,9.987239e-01,2.467499e-09,4.360570e-15,6.736431e-02,1.857957e-05,4.284354e-05,2.917051e-04,1.000000,6.988458e-05,2.089071e-05,2.443423e-12,1.580119e-04
61189,0.002830,8.756841e-13,8.690323e-12,1.376859e-15,6.557957e-17,3.052384e-03,9.292792e-01,2.591388e-16,6.683469e-04,1.515746e-04,6.355450e-02,4.989203e-07,0.052851,1.732796e-03,2.021580e-13,8.547051e-18,5.542099e-03


In [37]:
preds = []
for i in tqdm(range(sub.shape[0]), miniters=1000):
    a = sub.loc[[i]]
    a = a.apply(lambda x: x > 0.2, axis=1)
    a = a.transpose()
    a= a.loc[a[i] == True]
    ' '.join(list(a.index))
    preds.append(' '.join(list(a.index)))

sam_sub['tags'] = preds
sam_sub.to_csv('subm.csv', index=False)

100%|██████████| 61191/61191 [01:53<00:00, 538.85it/s]


# Now, we have saved the result into the final submission file(csv)  